In [1]:
import requests
from bs4 import BeautifulSoup as bs
import yaml
from IPython.display import display
from time import sleep
import re



In [2]:
from typing import List, Dict, Any, Set, Tuple, Union, Optional

In [3]:
url = "http://factcheck.snu.ac.kr/v2/facts/%d"

In [11]:
responce = requests.get(url % 3640)

In [12]:
responce.status_code

200

In [14]:
soup = bs(responce.text, 'html.parser')

In [33]:
fc_items = soup.select(".fcItem_vf > ul:nth-child(1)")
for fc_item in fc_items:
    score = soup.select("script:nth-child(2)")
    print(score)

[<script src="/assets/truth-o-meter-51889745b0e9ff51b3aae488c87c3441a3d23f1cb5fbfd8ea8ea8940ca48a4e3.js"></script>, <script charset="utf-8" type="text/javascript">
  $(function () {
    showScore(12578, 1)
  });
</script>]


In [4]:
res149 = requests.get(url % 149)
soup149 = bs(res149.text, 'html.parser')

In [12]:

# SNU 팩트체크 사이트는 다음과 같이 팩트 체크 별 아이디와 점수를 보냅니다.
# <ul> <li class="fcItem_vf_li">...</li>
# <script charset="utf-8" type="text/javascript">
# $(function () { showScore(아이디, 점수)}); </script> ... </ul>
# 따라서 아이디와 점수를 추출하는 정규표현식을 사용합니다.

id_score_re = re.compile(r'(?<=showScore\()\d+, \d+')
  
# 아이디와 점수를 추출하는 함수를 정의합니다.

def get_fc_id_score(soup:bs) -> List[Tuple[int, int]]:
    id_score: List[Tuple[int,int]] = [] # 아이디와 점수를 저장할 리스트
    for fc_item_script in soup.select(".fcItem_vf > ul > script"): # 팩트 체크 아이템의 스크립트를 찾습니다.
        id_score_str: str = id_score_re.search(fc_item_script.text).group() # 정규 표현식으로 아이디와 점수값을 담은 문자열을 추출합니다.
        id_score_int: Tuple = tuple(map(int, id_score_str.split(', '))) # 문자열을 정수로 변환합니다.
        id_score.append(id_score_int)
    return id_score


whitespace_re = re.compile(r'\s{2,}') # 공백 제거용 정규표현식을 정의합니다.

# 작성 시간과 내용을 추출하는 함수를 정의합니다.

def get_fc_time_contents(soup:bs) -> List[Tuple[str, str, str]]:
    time_contents: List[str] = [] # 시간과 내용을 저장할 리스트
    for fc_item in soup.select(".fcItem_vf > ul > li"): # 팩트 체크 아이템을 찾습니다.
        date = fc_item.select_one(".reg_date > p i:nth-child(1)").text # 작성 날짜를 추출합니다.
        time = fc_item.select_one(".reg_date > p i:nth-child(2)").text # 작성 시간을 추출합니다.
        raw_content = fc_item.select_one(".vf_exp_wrap").text # 팩트 체크 내용을 추출합니다.
        content = re.sub(whitespace_re, " ", raw_content.strip()) # 공백을 제거합니다.
        time_contents.append((date, time, content))
    return time_contents


In [13]:
id_score = get_fc_id_score(soup149)
time_contents = get_fc_time_contents(soup149)

[('2017.04.28',
  '22:34',
  '검증내용 사실이 아니다. 지니계수는 소득분배 불평등을 나타내는 수치로 0에 가까울수록 소득 분배가 평등하고 1에 가까울수록 불평등함을 나타낸다.\xa0통계청 가계동향조사를 보면 노무현 정부 시기(2003년 2월~2008년 2월) 동안의 지니계수는 도시 2인이상 가구 시장소득 기준으로 갈수록 상승했다. 그러나 이명박 대통령이 취임한 해인 2008년 더 상승, 2009년 정점을 찍었다. 2006년부터 조사된 전체가구 시장소득 기준 지니계수도 이명박 정부 취임 이후에 더 올랐다. 시장소득이 아닌 처분가능 소득을 기준으로 해도 노무현 정부보다는 이명박 정부에서 지니계수가 더 높게 나타난다. 검증기사 [팩트파인더] 노무현 정부 때 지니계수 최악이었다?'),
 ('2017.04.29',
  '16:21',
  '검증내용 2인 이상 가구를 기준으로 했을 때 우리나라의 지니계수가 가장 높았던 시기는 2009년으로 노무현 정부(2003년 2월~ 2008년 2월)가 아닌,\xa0이명박 전 대통령 재임기간이다. 따라서 노무현 정부 때 지니계수가 가장 나빴다는 홍준표 후보의 발언은 틀린 말이다.\xa0하지만 노무현 정부 시절 지니계수가 꾸준히 악화된 게 사실이고, 전체가구 기준으로는 노 전 대통령과 이 전 대통령의 임기가 겹쳤던\xa02008년과 이 전 대통령 시절인 2009년의 수치가 같다. 따라서 노 전대통령의 경제정책 실패로 지수가 계속 상승해 이명박 정부 초반에 정점을 찍었다는 주장이라면 어느정도 홍 후보의 주장이 설득력이 있어 대체로 거짓이라고 할 수 있다. 검증기사 [팩트체크] 지니계수, 노무현 대통령 때 가장 높았다? 근거자료 1 : 링크(제목없음)'),
 ('2017.05.01',
  '15:09',
  '검증내용 지니계수(Gini coefficient)는?소득 불평등 정도를 나타내는 지니계수는 0에 가까우면 소득 분배가 평등하다는 의미이고, 1에 가깝게 높으면 불평등하다는 뜻이다. 즉 0의 수치는 완전한 평등(모

In [8]:
id_score_re = re.compile(r'(?<=showScore\()\d+, \d+')

In [11]:
# 팩트체크 작성 시간 및 내용을 추출
for fc_item in soup149.select(".fcItem_vf > ul > li"):
    print(fc_item.select_one(".reg_date > p i:nth-child(1)").text)
    print(fc_item.select_one(".reg_date > p i:nth-child(2)").text)
    content = fc_item.select_one(".vf_exp_wrap").text
    print(re.sub(whitespace_re, " ", content.strip()))

2017.04.28
22:34
검증내용 사실이 아니다. 지니계수는 소득분배 불평등을 나타내는 수치로 0에 가까울수록 소득 분배가 평등하고 1에 가까울수록 불평등함을 나타낸다. 통계청 가계동향조사를 보면 노무현 정부 시기(2003년 2월~2008년 2월) 동안의 지니계수는 도시 2인이상 가구 시장소득 기준으로 갈수록 상승했다. 그러나 이명박 대통령이 취임한 해인 2008년 더 상승, 2009년 정점을 찍었다. 2006년부터 조사된 전체가구 시장소득 기준 지니계수도 이명박 정부 취임 이후에 더 올랐다. 시장소득이 아닌 처분가능 소득을 기준으로 해도 노무현 정부보다는 이명박 정부에서 지니계수가 더 높게 나타난다. 검증기사 [팩트파인더] 노무현 정부 때 지니계수 최악이었다?
2017.04.29
16:21
검증내용 2인 이상 가구를 기준으로 했을 때 우리나라의 지니계수가 가장 높았던 시기는 2009년으로 노무현 정부(2003년 2월~ 2008년 2월)가 아닌, 이명박 전 대통령 재임기간이다. 따라서 노무현 정부 때 지니계수가 가장 나빴다는 홍준표 후보의 발언은 틀린 말이다. 하지만 노무현 정부 시절 지니계수가 꾸준히 악화된 게 사실이고, 전체가구 기준으로는 노 전 대통령과 이 전 대통령의 임기가 겹쳤던 2008년과 이 전 대통령 시절인 2009년의 수치가 같다. 따라서 노 전대통령의 경제정책 실패로 지수가 계속 상승해 이명박 정부 초반에 정점을 찍었다는 주장이라면 어느정도 홍 후보의 주장이 설득력이 있어 대체로 거짓이라고 할 수 있다. 검증기사 [팩트체크] 지니계수, 노무현 대통령 때 가장 높았다? 근거자료 1 : 링크(제목없음)
2017.05.01
15:09
검증내용 지니계수(Gini coefficient)는?소득 불평등 정도를 나타내는 지니계수는 0에 가까우면 소득 분배가 평등하다는 의미이고, 1에 가깝게 높으면 불평등하다는 뜻이다. 즉 0의 수치는 완전한 평등(모두가 정확히 동일한 소득을 가지)을, 1의 수치는 완전한 불평등을 의미(한사람이 전체소득을, 나머지 사